In [1]:
import pip

package_names = ['awswrangler'] 
pip.main(['install'] + package_names + ['--upgrade'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


0

In [2]:
# Nativos
from dateutil.relativedelta import relativedelta
from time import gmtime, strftime
from datetime import datetime
import random as rn
import joblib
import json
import sys
import os
import gc

#nube
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker import get_execution_role
import awswrangler as wr
import sagemaker
import boto3

#calculo
import pandas as pd
import numpy as np
import scipy

#grafico
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid")

#Interacciones con output
import warnings
warnings.filterwarnings("ignore")
# warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

gc.collect()
# MODELS
#from lightgbm import LGBMClassifier

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)
#import scorecardpy as sc
#from utils import *
#from graphs import *

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

#from utils import *
#from modeler import *
#subfolder = "data"
# os.listdir(subfolder)

from setup import *

////////////////////////////////////////////////// 

path_container_input:  /opt/ml/processing/input
path_container_output:  /opt/ml/processing/output
path_container_utils:  /opt/ml/processing/input/utils
path_container_universo:  /opt/ml/processing/input/HM_UNIVERSO_PROPENSION_DESEMBOLSO_BPE
target :::::::::  target_desembolso_f2m_mayor_30_menor_180


In [3]:
# imports
from UTILITARIO_CODE.utils import targets
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.amazon.amazon_estimator import get_image_uri
import awswrangler as wr
import pandas as pd
import sagemaker
import boto3

print("bucket: ", bucket)
print("region: ", region)
print("now: ", now)
print("sufijo: ", sufijo)

bucket:  sagemaker-us-east-1-058528764918
region:  us-east-1
now:  2021-09-27 18:59:56.870733
sufijo:  target_desembolso_f2m_mayor_30_menor_180_clasif_normal


In [4]:
print(len(seleccionadas))
print(seleccionadas)

19
['deuda_sf_prom_ult9m', 'saldo_coloc_direct_tc', 'prom_reprog_u12m', 'ult_var_saldo_ajustado_amt', 'nro_entid_financ_prom_ult9m_cnt', 'monto_adquirido_u6_amt', 'nro_var_10k_30k_negativa_u6', 'var_neta_saldo_ajustado_u3_amt', 'prom_gar_u12m', 'nroregs_reactiva_bcos', 'monto_pagado_u3_amt', 'saldo_coloc_direct_vig_bcos', 'saldo_coloc_direct_vig_cmpt', 'nro_entidades', 'monto_pagado_ult_rcc_amt', 'tendencia_nro_coloc_direct_bancos_v2', 'sow_cajas', 'prom_fae_u12m', 'nro_var_10k_30k_negativa_u3']


### Load data

In [5]:
print("uri_output: ", uri_output)
print("uri_contraste: ", uri_contraste)
print("uri_predict: ", uri_predict)

uri_output:  s3://sagemaker-us-east-1-058528764918/vpc/propension/output3
uri_contraste:  s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste
uri_predict:  s3://sagemaker-us-east-1-058528764918/vpc/propension/prediccion


** OJO: modificar esta parte en caso de cambios de periodos**

In [6]:
#uri_validation_202102 = uri_contraste + '/202102_{}.csv'.format(sufijo)
uri_validation_202103 = uri_contraste + '/202103_{}.csv'.format(sufijo)
uri_validation_202104 = uri_contraste + '/202104_{}.csv'.format(sufijo)
uri_validation_202105 = uri_contraste + '/202105_{}.csv'.format(sufijo)
uri_validation_202106 = uri_contraste + '/202106_{}.csv'.format(sufijo)
uri_validation_202107 = uri_contraste + '/202107_{}.csv'.format(sufijo)

#print(uri_validation_202102)
print(uri_validation_202103)
print(uri_validation_202104)
print(uri_validation_202105)
print(uri_validation_202106)
print(uri_validation_202107)

s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste/202103_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv
s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste/202104_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv
s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste/202105_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv
s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste/202106_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv
s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste/202107_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv


In [7]:
#_2006 = wr.s3.read_csv(
#    uri_validation_202106, 
#    sep=',',
#    dtype={'cod_sbs_val': str}
#)

#_2006.head()

In [8]:
# _2006.shape

### FEEDBACK

In [9]:
campania_ibk = pd.read_csv(
    'resultados_preaprobados_aporbados_5.csv', 
    dtype={'PERIODO': str, 'NUM_RUC': str, 'COD_SBS': str, 'CODSBS': str, 'COD_UNICO': str}
)

campania_ibk = campania_ibk[campania_ibk['CAMPANHA'] == 'Ecosistemas Corporativos']

campania_ibk.head()

,CODSBS,TIPDOC,NOMBRE_COMPLETO,TIPPERSONA,FLG_ACEPTA,FLG_GESTIONADO,FLG_DESEMBOLSADO,FLG_CONTACTADO,ID_EMPRESA,PERIODO,CAMPANHA,NUM_RUC,COD_UNICO,GESTION_TIPO_2,GESTION_SUBTIPO,GESTION_COMENTARIO,GESTION_FECHA_REGISTRO,GESTIONADO,Contactados,Acepta,Desembolsado,CTD_DESEMBOLSO,CTD_DESPLEGADOS,COLOCACION_NETA,FRESCURA_LEAD,OFERTA_APROBADA,flg_tlv,Acepta_EN,Acepta_tlv,Acepta_tlv_sin_contacto,BARRIDO,CE,INTERESADO,Pasa_pre_filtro,PASA_OFERTA,ENVIADO_GTP,TIPO_PERSONA,ID_CAMPANHA_INSTANCIA,canal,registro_ejecutivo,SUB_CAMPANHA,GESTION_TIPO_TLV,GESTION_TLV_NIVEL_1,GESTION_TLV_NIVEL_2,CANAL_INICIAL,FLG_REPROGRAMACION,LETTER,PERIODO_CAMPANHA,COD_SBS
22,0204780579,1.0,LOPEZ . CATALINA JACINTA\r,1.0,0,0,0,0,3031270.0,202109,Ecosistemas Corporativos,10002131426,0010782493,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,NaN,Repetido,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,77.0,NaN,NaN,Ecosistemas Corporativos,NaN,NaN,NaN,TLV,NaN,10,202109,0204780579
23,0136744097,1.0,HUANACO DE PAXI BERNA .,1.0,0,0,0,0,2902127.0,202106,Ecosistemas Corporativos,10004011053,NaN,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,NaN,Fresco,120000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,54.0,NaN,NaN,Ecosistemas Corporativos,NaN,NaN,NaN,NaN,NaN,10,202106,0136744097
24,0136744097,1.0,HUANACO DE PAXI BERNA .,1.0,0,0,0,0,2902127.0,202108,Ecosistemas Corporativos,10004011053,NaN,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,NaN,Repetido,120000.0,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,PN,69.0,NaN,NaN,Ecosistemas Corporativos,Sin Contacto,A.NO CONTACTO,A.NO CONTACTO,TLV,NaN,10,202108,0136744097
25,0136744097,1.0,HUANACO DE PAXI BERNA .,1.0,0,0,0,0,2902127.0,202107,Ecosistemas Corporativos,10004011053,NaN,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,NaN,Repetido,120000.0,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,PN,61.0,NaN,NaN,Ecosistemas Corporativos,Sin Contacto,A.TELEFONO ERRADO,A.TELEFONO ERRADO,EN,NaN,10,202107,0136744097
39,0204780579,1.0,LOPEZ . CATALINA JACINTA\r,1.0,0,0,0,0,3031270.0,202108,Ecosistemas Corporativos,10002131426,0010782493,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,NaN,Fresco,120000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,69.0,NaN,NaN,Ecosistemas Corporativos,NaN,NaN,NaN,TLV,NaN,10,202108,0204780579


In [10]:
campania_ibk.isnull().sum()

CODSBS                      26080
TIPDOC                      26080
NOMBRE_COMPLETO             26414
TIPPERSONA                  26080
FLG_ACEPTA                      0
FLG_GESTIONADO                  0
FLG_DESEMBOLSADO                0
FLG_CONTACTADO                  0
ID_EMPRESA                      1
PERIODO                         0
CAMPANHA                        0
NUM_RUC                         0
COD_UNICO                   65375
GESTION_TIPO_2             115736
GESTION_SUBTIPO            115974
GESTION_COMENTARIO         116033
GESTION_FECHA_REGISTRO     115747
GESTIONADO                      0
Contactados                     0
Acepta                     121789
Desembolsado                    0
CTD_DESEMBOLSO                  0
CTD_DESPLEGADOS                 0
COLOCACION_NETA            122054
FRESCURA_LEAD                   0
OFERTA_APROBADA                 0
flg_tlv                     61061
Acepta_EN                  121795
Acepta_tlv                  62187
Acepta_tlv_sin

In [11]:
campania_ibk[campania_ibk['COD_SBS'].isnull()].groupby(by=['PERIODO']).size()

PERIODO
202105    2236
202106    3654
202107    3355
202108    8992
202109    7843
dtype: int64

In [12]:
print(campania_ibk.shape)
campania_ibk = campania_ibk[campania_ibk['COD_SBS'].notnull()]
campania_ibk['COLOCACION_NETA'] = campania_ibk['COLOCACION_NETA'].fillna(0)                        
print(campania_ibk.shape)

campania_ibk.head()

(122399, 49)
(96319, 49)


,CODSBS,TIPDOC,NOMBRE_COMPLETO,TIPPERSONA,FLG_ACEPTA,FLG_GESTIONADO,FLG_DESEMBOLSADO,FLG_CONTACTADO,ID_EMPRESA,PERIODO,CAMPANHA,NUM_RUC,COD_UNICO,GESTION_TIPO_2,GESTION_SUBTIPO,GESTION_COMENTARIO,GESTION_FECHA_REGISTRO,GESTIONADO,Contactados,Acepta,Desembolsado,CTD_DESEMBOLSO,CTD_DESPLEGADOS,COLOCACION_NETA,FRESCURA_LEAD,OFERTA_APROBADA,flg_tlv,Acepta_EN,Acepta_tlv,Acepta_tlv_sin_contacto,BARRIDO,CE,INTERESADO,Pasa_pre_filtro,PASA_OFERTA,ENVIADO_GTP,TIPO_PERSONA,ID_CAMPANHA_INSTANCIA,canal,registro_ejecutivo,SUB_CAMPANHA,GESTION_TIPO_TLV,GESTION_TLV_NIVEL_1,GESTION_TLV_NIVEL_2,CANAL_INICIAL,FLG_REPROGRAMACION,LETTER,PERIODO_CAMPANHA,COD_SBS
22,0204780579,1.0,LOPEZ . CATALINA JACINTA\r,1.0,0,0,0,0,3031270.0,202109,Ecosistemas Corporativos,10002131426,0010782493,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,0.0,Repetido,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,77.0,NaN,NaN,Ecosistemas Corporativos,NaN,NaN,NaN,TLV,NaN,10,202109,0204780579
23,0136744097,1.0,HUANACO DE PAXI BERNA .,1.0,0,0,0,0,2902127.0,202106,Ecosistemas Corporativos,10004011053,NaN,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,0.0,Fresco,120000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,54.0,NaN,NaN,Ecosistemas Corporativos,NaN,NaN,NaN,NaN,NaN,10,202106,0136744097
24,0136744097,1.0,HUANACO DE PAXI BERNA .,1.0,0,0,0,0,2902127.0,202108,Ecosistemas Corporativos,10004011053,NaN,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,0.0,Repetido,120000.0,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,PN,69.0,NaN,NaN,Ecosistemas Corporativos,Sin Contacto,A.NO CONTACTO,A.NO CONTACTO,TLV,NaN,10,202108,0136744097
25,0136744097,1.0,HUANACO DE PAXI BERNA .,1.0,0,0,0,0,2902127.0,202107,Ecosistemas Corporativos,10004011053,NaN,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,0.0,Repetido,120000.0,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,PN,61.0,NaN,NaN,Ecosistemas Corporativos,Sin Contacto,A.TELEFONO ERRADO,A.TELEFONO ERRADO,EN,NaN,10,202107,0136744097
39,0204780579,1.0,LOPEZ . CATALINA JACINTA\r,1.0,0,0,0,0,3031270.0,202108,Ecosistemas Corporativos,10002131426,0010782493,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,0.0,Fresco,120000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,69.0,NaN,NaN,Ecosistemas Corporativos,NaN,NaN,NaN,TLV,NaN,10,202108,0204780579


In [13]:
from sklearn.metrics import roc_auc_score

In [14]:
def get_score_campania(mes_campania, data_rcc):
    camapania_target = campania_ibk[
        (campania_ibk['PERIODO'] == mes_campania)
    ]
    print(camapania_target.shape, camapania_target['PERIODO'].unique())
    
    try:
        print("-"*25)
        print("AUC BPE RCC:     ", roc_auc_score(data_rcc[target], data_rcc['score_desembolso']))
    except:
        pass
    
    camapania_target['COD_SBS'] = camapania_target['COD_SBS'].astype(int).astype(str)
    data_rcc['cod_sbs_val'] = data_rcc['cod_sbs_val'].astype(int).astype(str)
    
    validation_campania = data_rcc[data_rcc['cod_sbs_val'].isin(list(camapania_target['COD_SBS']))]
    try:
        roc_auc_campania = roc_auc_score(validation_campania[target], validation_campania['score_desembolso'])
        print("AUC CAMPANIA RCC: ", roc_auc_campania, roc_auc_campania*2-1)
    except:
        pass
    
    print("validation_campania: ", validation_campania.shape)
    validation_campania = validation_campania.merge(
        camapania_target, how='left', left_on='cod_sbs_val' , right_on ='COD_SBS'
    )
    print("camapania_target: ", camapania_target.shape)
    print("validation_campania: ", validation_campania.shape)
    
    roc_auc_acepta = roc_auc_score(validation_campania['FLG_ACEPTA'], validation_campania['score_desembolso'])
    print("AUC ACEPTA:      ",roc_auc_acepta, roc_auc_acepta*2-1)

    roc_auc_desembolsa = roc_auc_score(validation_campania['FLG_DESEMBOLSADO'], validation_campania['score_desembolso'])
    print("AUC DESEMBOLSA: ", roc_auc_desembolsa, roc_auc_desembolsa*2-1)
    
    group_ = validation_campania.groupby(by=['GRUPO_PROPENSION']).agg({
        target: ['count', 'sum', 'mean'], 
        'score_desembolso': 'mean',
        'FLG_ACEPTA': ['mean', 'sum'],
        'FLG_DESEMBOLSADO': ['mean', 'sum']
    })
    
    return group_, validation_campania

### 202102 ---> CAMPAÑA 202104

In [15]:
#_2002 = wr.s3.read_csv(
#    uri_validation_202102, 
#    sep=',',
##    dtype={'cod_sbs_val': str}
#)
#_2002.head(3)

In [16]:
#group_202104, campania_bpe_202104 = get_score_campania('202104', _2002)
#group_202104

In [17]:
#del _2002
#campania_bpe_202104['GRUPO_PROPENSION'].value_counts(normalize=True)*100

### 202103 ---> CAMPAÑA 202105

In [18]:
_2003 = wr.s3.read_csv(
    uri_validation_202103, 
    sep=',',
    dtype={'cod_sbs_val': str}
)
_2003.head(3)

,cod_sbs_val,target_desembolso_f2m_mayor_30_menor_180,deuda_sf_prom_ult9m,saldo_coloc_direct_tc,prom_reprog_u12m,ult_var_saldo_ajustado_amt,nro_entid_financ_prom_ult9m_cnt,monto_adquirido_u6_amt,nro_var_10k_30k_negativa_u6,var_neta_saldo_ajustado_u3_amt,prom_gar_u12m,nroregs_reactiva_bcos,monto_pagado_u3_amt,saldo_coloc_direct_vig_bcos,saldo_coloc_direct_vig_cmpt,nro_entidades,monto_pagado_ult_rcc_amt,tendencia_nro_coloc_direct_bancos_v2,sow_cajas,prom_fae_u12m,nro_var_10k_30k_negativa_u3,score_desembolso,GRUPO_PROPENSION
0,48759904,0.0,6077.54,0.0,4558.153333,-616.17,1,0.0,0,-1794.37,0.00,0,1794.37,0.0,0.0,1,616.17,0.000000,1.0,0.000,0,0.007450,I
1,149898271,0.0,24525.75,0.0,17737.965000,900.00,1,900.0,1,-23266.88,2.25,0,24166.88,0.0,0.0,1,0.00,0.333333,1.0,0.000,1,0.030875,C
2,126112726,0.0,7000.35,0.0,0.000000,0.00,1,0.0,0,0.00,1.00,0,0.00,0.0,0.0,1,0.00,1.000000,0.0,5833.625,0,0.007600,I


In [19]:
group_202105, campania_bpe_202105 = get_score_campania('202105', _2003)
group_202105

(10350, 49) ['202105']
-------------------------
AUC BPE RCC:      0.7720062554415492
AUC CAMPANIA RCC:  0.6454935289424697 0.2909870578849394
validation_campania:  (2450, 23)
camapania_target:  (10350, 49)
validation_campania:  (2450, 72)
AUC ACEPTA:       0.5603304117172183 0.12066082343443663
AUC DESEMBOLSA:  0.5189409650214296 0.037881930042859224


target_desembolso_f2m_mayor_30_menor_180                 score_desembolso FLG_ACEPTA     FLG_DESEMBOLSADO    
                                                    count   sum      mean             mean       mean sum             mean sum
GRUPO_PROPENSION                                                                                                              
A                                                     324  37.0  0.114198         0.091001   0.040123  13         0.027778   9
B                                                     327  19.0  0.058104         0.047458   0.024465   8         0.015291   5
C                                                     323  14.0  0.043344         0.033440   0.030960  10         0.015480   5
D                                                     274  12.0  0.043796         0.025144   0.014599   4         0.007299   2
E                                                     269   6.0  0.022305         0.020086   0.014870   4         0.011152   3
F                                                     306  10.0  0.032680         0.015630   0.016340   5         0.009804   3
G                                                     233   9.0  0.038627         0.012417   0.021459   5         0.012876   3
H                                                     182   2.0  0.010989         0.010045   0.005495   1         0.005495   1
I                                                     136   0.0  0.000000         0.007620   0.014706   2         0.014706   2
J                                                      76   7.0  0.092105         0.005279   0.078947   6         0.078947   6

In [20]:
del _2003
campania_bpe_202105['GRUPO_PROPENSION'].value_counts(normalize=True)*100

NumExpr defaulting to 4 threads.


B    13.346939
A    13.224490
C    13.183673
F    12.489796
D    11.183673
E    10.979592
G     9.510204
H     7.428571
I     5.551020
J     3.102041
Name: GRUPO_PROPENSION, dtype: float64

### 202104 ---> CAMPAÑA 202106

In [21]:
_2004 = wr.s3.read_csv(
    uri_validation_202104, 
    sep=',',
    dtype={'cod_sbs_val': str}
)

_2004.head(3)

,cod_sbs_val,target_desembolso_f2m_mayor_30_menor_180,deuda_sf_prom_ult9m,saldo_coloc_direct_tc,prom_reprog_u12m,ult_var_saldo_ajustado_amt,nro_entid_financ_prom_ult9m_cnt,monto_adquirido_u6_amt,nro_var_10k_30k_negativa_u6,var_neta_saldo_ajustado_u3_amt,prom_gar_u12m,nroregs_reactiva_bcos,monto_pagado_u3_amt,saldo_coloc_direct_vig_bcos,saldo_coloc_direct_vig_cmpt,nro_entidades,monto_pagado_ult_rcc_amt,tendencia_nro_coloc_direct_bancos_v2,sow_cajas,prom_fae_u12m,nro_var_10k_30k_negativa_u3,score_desembolso,GRUPO_PROPENSION
0,203570678,0.0,80314.40,0.0,0.0000,-9379.89,0,250000.00,0,231724.84,0.000000,0,18275.16,231724.84,231724.84,1,9379.89,0.0,0.000,0.000000,0,0.031269,C
1,183894013,0.0,25441.24,0.0,0.0000,0.00,0,0.00,0,0.00,0.000000,1,0.00,0.00,0.00,1,0.00,1.0,0.000,0.000000,0,0.006905,I
2,74279325,0.0,99463.00,0.0,81768.9575,-1384.21,1,10225.52,0,-4265.70,0.833333,0,4265.70,8600.59,0.00,2,1384.21,1.0,0.883,18727.464167,0,0.033284,C


In [22]:
group_202106, campania_bpe_202106 = get_score_campania('202106', _2004)
group_202106

(12461, 49) ['202106']
-------------------------
AUC BPE RCC:      0.7526423472331193
AUC CAMPANIA RCC:  0.6714062874970003 0.34281257499400053
validation_campania:  (2886, 23)
camapania_target:  (12461, 49)
validation_campania:  (2886, 72)
AUC ACEPTA:       0.6397837470392675 0.2795674940785351
AUC DESEMBOLSA:  0.6465496668074091 0.2930993336148182


target_desembolso_f2m_mayor_30_menor_180                 score_desembolso FLG_ACEPTA     FLG_DESEMBOLSADO    
                                                    count   sum      mean             mean       mean sum             mean sum
GRUPO_PROPENSION                                                                                                              
A                                                     335  22.0  0.065672         0.086409   0.023881   8         0.011940   4
B                                                     398  28.0  0.070352         0.046303   0.010050   4         0.007538   3
C                                                     362  17.0  0.046961         0.033292   0.013812   5         0.011050   4
D                                                     342  11.0  0.032164         0.025194   0.017544   6         0.005848   2
E                                                     357  15.0  0.042017         0.019874   0.011204   4         0.008403   3
F                                                     359   7.0  0.019499         0.015584   0.008357   3         0.002786   1
G                                                     269   1.0  0.003717         0.012511   0.003717   1         0.003717   1
H                                                     221   6.0  0.027149         0.010107   0.009050   2         0.004525   1
I                                                     149   1.0  0.006711         0.007663   0.000000   0         0.000000   0
J                                                      94   0.0  0.000000         0.005247   0.000000   0         0.000000   0

In [23]:
del _2004
campania_bpe_202106['GRUPO_PROPENSION'].value_counts(normalize=True)*100

B    13.790714
C    12.543313
F    12.439362
E    12.370062
D    11.850312
A    11.607762
G     9.320859
H     7.657658
I     5.162855
J     3.257103
Name: GRUPO_PROPENSION, dtype: float64

### 202105 ---> CAMPAÑA 202107

In [24]:
_2005 = wr.s3.read_csv(
    uri_validation_202105, 
    sep=',',
    dtype={'cod_sbs_val': str}
)

_2005.head(3)

,cod_sbs_val,target_desembolso_f2m_mayor_30_menor_180,deuda_sf_prom_ult9m,saldo_coloc_direct_tc,prom_reprog_u12m,ult_var_saldo_ajustado_amt,nro_entid_financ_prom_ult9m_cnt,monto_adquirido_u6_amt,nro_var_10k_30k_negativa_u6,var_neta_saldo_ajustado_u3_amt,prom_gar_u12m,nroregs_reactiva_bcos,monto_pagado_u3_amt,saldo_coloc_direct_vig_bcos,saldo_coloc_direct_vig_cmpt,nro_entidades,monto_pagado_ult_rcc_amt,tendencia_nro_coloc_direct_bancos_v2,sow_cajas,prom_fae_u12m,nro_var_10k_30k_negativa_u3,score_desembolso,GRUPO_PROPENSION,GRUPO_PROPENSION_FINO
0,30419162,0.0,59487.23,0.0,0.000000,-2558.09,1,11789.22,0,7862.12,2.155599e+04,1,3927.10,43772.33,0.00,1,2558.09,1.000,0.000,0.0,0,0.033920,C,E
1,75300956,0.0,589446.28,0.0,328314.754167,-22543.07,2,633912.87,1,122979.29,1.162966e+06,0,510933.58,629297.28,629297.28,3,22543.07,0.875,0.000,0.0,1,0.077572,A,C
2,69166024,0.0,33044.68,0.0,15639.696667,-958.23,2,0.00,0,-3224.89,2.000000e+00,0,3224.89,28798.82,0.00,2,958.23,1.000,0.001,0.0,0,0.008842,I,I


In [25]:
group_202107, campania_bpe_202107 = get_score_campania('202107', _2005)
group_202107

(13670, 49) ['202107']
-------------------------
AUC BPE RCC:      0.76016894021159
AUC CAMPANIA RCC:  0.6591694067994756 0.3183388135989511
validation_campania:  (3729, 24)
camapania_target:  (13670, 49)
validation_campania:  (3729, 73)
AUC ACEPTA:       0.5568787931333449 0.11375758626668975
AUC DESEMBOLSA:  0.5646534384343571 0.12930687686871423


target_desembolso_f2m_mayor_30_menor_180                 score_desembolso FLG_ACEPTA     FLG_DESEMBOLSADO    
                                                    count   sum      mean             mean       mean sum             mean sum
GRUPO_PROPENSION                                                                                                              
A                                                     463  58.0  0.125270         0.093692   0.036717  17         0.021598  10
B                                                     494  29.0  0.058704         0.048476   0.022267  11         0.014170   7
C                                                     464  25.0  0.053879         0.034358   0.015086   7         0.010776   5
D                                                     445  16.0  0.035955         0.025885   0.017978   8         0.011236   5
E                                                     453  24.0  0.052980         0.020343   0.022075  10         0.017660   8
F                                                     445  18.0  0.040449         0.016024   0.017978   8         0.011236   5
G                                                     355  10.0  0.028169         0.013084   0.025352   9         0.016901   6
H                                                     273   5.0  0.018315         0.010356   0.010989   3         0.003663   1
I                                                     206   4.0  0.019417         0.007804   0.024272   5         0.014563   3
J                                                     131   2.0  0.015267         0.005312   0.007634   1         0.000000   0

In [26]:
_2006 = wr.s3.read_csv(
    uri_validation_202106, 
    sep=',',
    dtype={'cod_sbs_val': str}
)

_2006.head(3)

,cod_sbs_val,target_desembolso_f2m_mayor_30_menor_180,deuda_sf_prom_ult9m,saldo_coloc_direct_tc,prom_reprog_u12m,ult_var_saldo_ajustado_amt,nro_entid_financ_prom_ult9m_cnt,monto_adquirido_u6_amt,nro_var_10k_30k_negativa_u6,var_neta_saldo_ajustado_u3_amt,prom_gar_u12m,nroregs_reactiva_bcos,monto_pagado_u3_amt,saldo_coloc_direct_vig_bcos,saldo_coloc_direct_vig_cmpt,nro_entidades,monto_pagado_ult_rcc_amt,tendencia_nro_coloc_direct_bancos_v2,sow_cajas,prom_fae_u12m,nro_var_10k_30k_negativa_u3,score_desembolso,GRUPO_PROPENSION,GRUPO_PROPENSION_FINO
0,154769137,0.0,21749.62,0.00,1008.690000,-642.76,1,0.00,0,-1853.73,3.250,1,1853.73,8950.36,0.00,1,642.76,1.000000,0.000,0.0,0,0.005127,J,J
1,157683209,0.0,113613.00,85641.88,88991.732500,-2413.69,1,21148.22,0,7293.25,0.000,1,4374.28,91387.35,91387.35,1,2413.69,1.166667,0.000,0.0,0,0.016001,F,F
2,136905961,0.0,30885.53,0.00,6327.621667,11925.22,2,15233.98,0,11129.38,5448.325,1,4104.60,9445.35,0.00,2,854.97,1.000000,0.626,0.0,0,0.005851,J,J


In [27]:
group_202108, campania_bpe_202108 = get_score_campania('202108', _2006)
group_202108

(26819, 49) ['202108']
-------------------------
validation_campania:  (5776, 24)
camapania_target:  (26819, 49)
validation_campania:  (5776, 73)
AUC ACEPTA:       0.5744095950750943 0.14881919015018852
AUC DESEMBOLSA:  0.5815984345903166 0.16319686918063314


target_desembolso_f2m_mayor_30_menor_180           score_desembolso FLG_ACEPTA     FLG_DESEMBOLSADO    
                                                    count  sum mean             mean       mean sum             mean sum
GRUPO_PROPENSION                                                                                                        
A                                                     761  0.0  0.0         0.089872   0.015769  12         0.013141  10
B                                                     791  0.0  0.0         0.048379   0.015171  12         0.011378   9
C                                                     735  0.0  0.0         0.034235   0.006803   5         0.002721   2
D                                                     663  0.0  0.0         0.025775   0.012066   8         0.007541   5
E                                                     666  0.0  0.0         0.020267   0.021021  14         0.016517  11
F                                                     679  0.0  0.0         0.015787   0.008837   6         0.007364   5
G                                                     558  0.0  0.0         0.012928   0.008961   5         0.005376   3
H                                                     390  0.0  0.0         0.010174   0.002564   1         0.000000   0
I                                                     335  0.0  0.0         0.007771   0.002985   1         0.002985   1
J                                                     198  0.0  0.0         0.005177   0.010101   2         0.010101   2

In [28]:
_2007 = wr.s3.read_csv(
    uri_validation_202107, 
    sep=',',
    dtype={'cod_sbs_val': str}
)

_2007.head(3)

,cod_sbs_val,target_desembolso_f2m_mayor_30_menor_180,deuda_sf_prom_ult9m,saldo_coloc_direct_tc,prom_reprog_u12m,ult_var_saldo_ajustado_amt,nro_entid_financ_prom_ult9m_cnt,monto_adquirido_u6_amt,nro_var_10k_30k_negativa_u6,var_neta_saldo_ajustado_u3_amt,prom_gar_u12m,nroregs_reactiva_bcos,monto_pagado_u3_amt,saldo_coloc_direct_vig_bcos,saldo_coloc_direct_vig_cmpt,nro_entidades,monto_pagado_ult_rcc_amt,tendencia_nro_coloc_direct_bancos_v2,sow_cajas,prom_fae_u12m,nro_var_10k_30k_negativa_u3,score_desembolso,GRUPO_PROPENSION,GRUPO_PROPENSION_FINO
0,67313267,0.0,182353.38,0.00,0.000000,0.00,1,0.00,0,0.00,0.0,1,0.00,0.01,0.01,1,0.00,1.0,0.000,0.0,0,0.015278,F,F
1,28947739,0.0,45091.08,0.00,7515.042500,-1701.57,1,21000.00,0,16968.41,0.0,0,4031.59,38195.57,0.00,2,1701.57,1.0,0.343,0.0,0,0.029214,D,E
2,53816525,0.0,842.23,16.99,723.640833,-2865.61,0,3501.56,0,-183.01,0.0,0,3020.67,16.99,16.99,1,2865.61,1.5,0.000,0.0,0,0.011504,H,H


In [29]:
group_202109, campania_bpe_202109 = get_score_campania('202109', _2007)
group_202109

(27617, 49) ['202109']
-------------------------
validation_campania:  (6335, 24)
camapania_target:  (27617, 49)
validation_campania:  (6335, 73)
AUC ACEPTA:       0.5594988662131519 0.11899773242630385
AUC DESEMBOLSA:  0.7010583056565977 0.4021166113131953


target_desembolso_f2m_mayor_30_menor_180           score_desembolso FLG_ACEPTA     FLG_DESEMBOLSADO    
                                                    count  sum mean             mean       mean sum             mean sum
GRUPO_PROPENSION                                                                                                        
A                                                     859  0.0  0.0         0.095796   0.011641  10         0.004657   4
B                                                     817  0.0  0.0         0.049980   0.001224   1         0.001224   1
C                                                     769  0.0  0.0         0.035401   0.007802   6         0.005202   4
D                                                     736  0.0  0.0         0.026821   0.005435   4         0.001359   1
E                                                     727  0.0  0.0         0.020963   0.004127   3         0.000000   0
F                                                     774  0.0  0.0         0.016386   0.003876   3         0.001292   1
G                                                     650  0.0  0.0         0.013411   0.001538   1         0.000000   0
H                                                     442  0.0  0.0         0.010504   0.000000   0         0.000000   0
I                                                     351  0.0  0.0         0.007859   0.014245   5         0.002849   1
J                                                     210  0.0  0.0         0.005314   0.009524   2         0.000000   0

#### ENOQUE DE ENTRENAMIENTO ANTIGUO